In [1]:
import yaml
import requests
import pandas as pd
import json
import sqlalchemy
import pg8000
from sqlalchemy import Table, Column, Integer, String, MetaData, Date, Boolean, Float
from sqlalchemy.sql import select, insert
import time
import os
import time
from collections import ChainMap


In [2]:

with open('../../footballYaml.yml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
football_key = config['football_key']
db_user = config["db_user"]
db_pass = config["db_pass"]
db_name = config["db_name"]
db_host = config["db_host"]

# Functions

### Function to get the results by seasons.

* The function needs two parameters: season (yearly) and league id.
* Premier League's id is 39.

In [3]:
# Funcion creada Nelson 
def compare_result(a,b):
    if None not in (a, b):
        if b > a:
          c ="lose"
        elif a == b:
          c = "draw"
        else:
          c = "win"
    else: 
        c = None
    return c
compare_result(2,None)

In [4]:
# Funcion modificada Nelson 
def season_data(year):
    url = "https://v3.football.api-sports.io/fixtures"
    querystring = {"league":"39","season":year}
    payload={}
    headers = {
        'x-rapidapi-key': football_key,
        'x-rapidapi-host': 'v3.football.api-sports.io'
    }
    response = requests.request("GET", url, headers=headers, params=querystring,  data=payload).json()
    season_data = []
    for row in response["response"]:
        season_data.append({"season":row["league"]["season"],
                               "match_date":row["fixture"]["date"],
                               "match_id":row["fixture"]["id"],
                               "local_team":row["teams"]["home"]["name"],
                                "local_team_id":row["teams"]["home"]["id"],
                               "away_team":row["teams"]["away"]["name"],
                                "away_team_id":row["teams"]["away"]["id"],
                               "local_goals":row["goals"]["home"],
                               "away_goals":row["goals"]["away"],
                               "local_result":compare_result(row["goals"]["home"],row["goals"]["away"])})
    return season_data


In [5]:
def season_data(year):
    url = "https://v3.football.api-sports.io/fixtures"
    querystring = {"league":"39","season":year}
    payload={}
    headers = {
        'x-rapidapi-key': football_key,
        'x-rapidapi-host': 'v3.football.api-sports.io'
    }
    response = requests.request("GET", url, headers=headers, params=querystring,  data=payload).json()
    season_data = []
    for row in response["response"]:
        season_data.append({"season":row["league"]["season"],
                               "match_date":row["fixture"]["date"],
                               "match_id":row["fixture"]["id"],
                               "local_team":row["teams"]["home"]["name"],
                                "local_team_id":row["teams"]["home"]["id"],
                               "away_team":row["teams"]["away"]["name"],
                                "away_team_id":row["teams"]["away"]["id"],
                               "local_goals":row["goals"]["home"],
                               "away_goals":row["goals"]["away"]})
    return season_data


### Function to get relevant statistic by player, team and seasons. 


* This endpoint returns a player's data and statistics available. Note that it is possible that a player has statistics for two teams in the same season. In that case the key is a combination of team id and player id
* The statistics are calculated according to the team id, league id and season.
* The player's id are unique in the API.
* This endpoint uses a pagination system, you can navigate between the different pages thanks to the page parameter.
* The season 2019 has 33 pages, season 2020 has 37 pages and season 2021 has 39 pages.
* One request per page is allowed with the free plan 

In [13]:
def player_stats(year):
    url = "https://v3.football.api-sports.io/players"
    querystring = {"league":"39","season":year}
    
    headers = {
        'x-rapidapi-host': "v3.football.api-sports.io",
        'x-rapidapi-key': football_key
    }
    
    response = requests.request("GET", url, headers=headers, params=querystring).json()
    statistic_data = []
    for row in response["response"]:
        statistic_data.append({"season":row["statistics"][0]["league"]["season"],
                               "player_id":row["player"]["id"],
                               "player_name":row["player"]["name"],
                               "age":row["player"]["age"],
                               "height":row["player"]["height"],
                               "weight":row["player"]["weight"],
                               "injured":row["player"]["injured"],
                               "team_id":row["statistics"][0]["team"]["id"],
                               "appearences":row["statistics"][0]["games"]["appearences"],
                               "minutes":row["statistics"][0]["games"]["minutes"],
                               "position":row["statistics"][0]["games"]["position"],
                               "rating":row["statistics"][0]["games"]["rating"],
                               "shots":row["statistics"][0]["shots"]["total"],
                               "goals":row["statistics"][0]["goals"]["total"],
                               "assists":row["statistics"][0]["goals"]["assists"],
                               "passes_accuracy":row["statistics"][0]["passes"]["accuracy"],
                               "total_duels":row["statistics"][0]["duels"]["total"],
                               "won_duels":row["statistics"][0]["duels"]["won"]
                              })
        
    return statistic_data

In [7]:
def floatsy(string):
    if string=='–' or isinstance(string, type(None)):
       return 5
    else:
       return float(string)

In [8]:
def statistics(match_id):
    url = "https://v3.football.api-sports.io/fixtures/players"

    querystring = {"fixture":match_id}
    payload={}

    headers = {
                'x-rapidapi-host': 'v3.football.api-sports.io',
                'x-rapidapi-key': football_key
                }
    
    response = requests.request("GET", url, headers=headers, params=querystring,  data=payload).json()
    local_players_data=[]
    
    row = response["response"][0]

    local_players_data.append({"match_id":response["parameters"]["fixture"],
                                 "team_id":row["team"]["id"],
                                    "local_player_0_id":row["players"][0]["player"]["id"],
                                    "local_player_0_rating":floatsy(row["players"][0]["statistics"][0]["games"]["rating"]),
                                    "local_player_1_id":row["players"][1]["player"]["id"],
                                    "local_player_1_rating":floatsy(row["players"][1]["statistics"][0]["games"]["rating"]),
                                    "local_player_2_id":row["players"][2]["player"]["id"],
                                    "local_player_2_rating":floatsy(row["players"][2]["statistics"][0]["games"]["rating"]),
                                    "local_player_3_id":row["players"][3]["player"]["id"],
                                    "local_player_3_rating":floatsy(row["players"][3]["statistics"][0]["games"]["rating"]),
                                    "local_player_4_id":row["players"][4]["player"]["id"],
                                    "local_player_4_rating":floatsy(row["players"][4]["statistics"][0]["games"]["rating"]),
                                    "local_player_5_id":row["players"][5]["player"]["id"],
                                    "local_player_5_rating":floatsy(row["players"][5]["statistics"][0]["games"]["rating"]),
                                    "local_player_6_id":row["players"][6]["player"]["id"],
                                    "local_player_6_rating":floatsy(row["players"][6]["statistics"][0]["games"]["rating"]),
                                    "local_player_7_id":row["players"][7]["player"]["id"],
                                    "local_player_7_rating":floatsy(row["players"][7]["statistics"][0]["games"]["rating"]),
                                    "local_player_8_id":row["players"][8]["player"]["id"],
                                    "local_player_8_rating":floatsy(row["players"][8]["statistics"][0]["games"]["rating"]),
                                    "local_player_9_id":row["players"][9]["player"]["id"],
                                    "local_player_9_rating":floatsy(row["players"][9]["statistics"][0]["games"]["rating"]),
                                    "local_player_10_id":row["players"][10]["player"]["id"],
                                    "local_player_10_rating":floatsy(row["players"][10]["statistics"][0]["games"]["rating"])})
    
    away_players_data=[]
    row = response["response"][1]

    away_players_data.append({"match_id":response["parameters"]["fixture"],
                                 "team_id":row["team"]["id"],
                                    "away_player_0_id":row["players"][0]["player"]["id"],
                                    "away_player_0_rating":floatsy(row["players"][0]["statistics"][0]["games"]["rating"]),
                                    "away_player_1_id":row["players"][1]["player"]["id"],
                                    "away_player_1_rating":floatsy(row["players"][1]["statistics"][0]["games"]["rating"]),
                                    "away_player_2_id":row["players"][2]["player"]["id"],
                                    "away_player_2_rating":floatsy(row["players"][2]["statistics"][0]["games"]["rating"]),
                                    "away_player_3_id":row["players"][3]["player"]["id"],
                                    "away_player_3_rating":floatsy(row["players"][3]["statistics"][0]["games"]["rating"]),
                                    "away_player_4_id":row["players"][4]["player"]["id"],
                                    "away_player_4_rating":floatsy(row["players"][4]["statistics"][0]["games"]["rating"]),
                                    "away_player_5_id":row["players"][5]["player"]["id"],
                                    "away_player_5_rating":floatsy(row["players"][5]["statistics"][0]["games"]["rating"]),
                                    "away_player_6_id":row["players"][6]["player"]["id"],
                                    "away_player_6_rating":floatsy(row["players"][6]["statistics"][0]["games"]["rating"]),
                                    "away_player_7_id":row["players"][7]["player"]["id"],
                                    "away_player_7_rating":floatsy(row["players"][7]["statistics"][0]["games"]["rating"]),
                                    "away_player_8_id":row["players"][8]["player"]["id"],
                                    "away_player_8_rating":floatsy(row["players"][8]["statistics"][0]["games"]["rating"]),
                                    "away_player_9_id":row["players"][9]["player"]["id"],
                                    "away_player_9_rating":floatsy(row["players"][9]["statistics"][0]["games"]["rating"]),
                                    "away_player_10_id":row["players"][10]["player"]["id"],
                                    "away_player_10_rating":floatsy(row["players"][10]["statistics"][0]["games"]["rating"])})
    
    
    return local_players_data,away_players_data
    

# Request to get information

We use the next code to get the information for the last 3 season of the Premier League

In [9]:
seasons =[2016,2017,2018,2019, 2020]
all_season_data = {}


for year in seasons:
    all_season_data[year]= season_data(year)

In [14]:
seasons =[2016,2017,2018,2019, 2020]
players_data = {}


for year in seasons:
    players_data[year]= player_stats(year)

In [10]:

local = []
away = []
for year in seasons:
    matches=[]
    for match in range(0,len(all_season_data[year])):
        matches.append(all_season_data[year][match]["match_id"])
     
    i=0
    for match in matches:
        loc, aw = statistics(match)       
        local.append({match:loc})
        away.append({match:aw})
        # 300 requests per minute so we need to stop it every once in a while
        i+=1
        if (i % 298 == 0):
            time.sleep(61)
            
with open("local", "w") as fp:
     json.dump(local, fp)
        
with open("away", "w") as fp:
     json.dump(away, fp)

        

# Set up data base

As we specify on the readme the idea is to use a postgres data base to manage and upload the data. So, we create a postreges instance on GCP and define the conextion as follow:

In [39]:
host_args = db_host.split(":")
if len(host_args) == 1:
    db_hostname = db_host
    db_port = 5432
elif len(host_args) == 2:
    db_hostname, db_port = host_args[0], int(host_args[1])

conn = sqlalchemy.create_engine(
    # Equivalent URL:
    # postgresql+pg8000://<db_user>:<db_pass>@<db_host>:<db_port>/<db_name>
    sqlalchemy.engine.url.URL.create(
        drivername="postgresql+pg8000",
        username=db_user,  # e.g. "my-database-user"
        password=db_pass,  # e.g. "my-database-password"
        host=db_hostname,  # e.g. "127.0.0.1"
        port=db_port,  # e.g. 5432
        database=db_name  # e.g. "my-database-name"
    )
)

**Test the connection**

In [40]:
conn.connect()

Define function to insert table on GCP

In [41]:
def insert_data(conn, table, data):
 ins = table.insert()
 conn.execute(ins, data)

We create two tables _match_ table and _statistic_ table.

In [42]:
meta = MetaData()

In [43]:
match = Table(
    'match', meta, 
    Column('season', Integer), 
    Column('match_date', Date),
    Column('match_id', Integer, primary_key = True), 
    Column('local_team', String),
    Column('local_team_id', Integer),
    Column('away_team', String),
    Column('away_team_id', Integer),
    Column('local_goals', Integer),
    Column('away_goals', Integer),
    Column('local_result', String)
)

In [44]:
# Nelson 
match2 = Table(
    'match2', meta, 
    Column('season', Integer), 
    Column('match_date', Date),
    Column('match_id', Integer, primary_key = True), 
    Column('local_team', String),
    Column('local_team_id', Integer),
    Column('away_team', String),
    Column('away_team_id', Integer),
    Column('local_goals', Integer),
    Column('away_goals', Integer),
    Column('local_result', String)
)

In [45]:
statistics = Table(
    'statistics', meta, 
    Column('season', Integer, primary_key = True), 
    Column('player_id', Integer, primary_key = True),
    Column('player_name', String),
    Column('age', Integer),
    Column('injured', Boolean, nullable=True),
    Column('team_id', Integer),
    Column('appearences', Integer, nullable=True),
    Column('minutes', Integer, nullable=True),
    Column('position', String, nullable=True),
    Column('rating', Float, nullable=True),
    Column('shots', Integer, nullable=True),
    Column('goals', Integer, nullable=True),
    Column('assists', Integer, nullable=True),
    Column('passes_accuracy', Float, nullable=True),
    Column('total_duels', Integer, nullable=True),
    Column('won_duels', Integer, nullable=True) 
)

In [46]:
#import json

#local = json.loads(open("local","r").read())
#away = json.loads(open("away","r").read())

In [47]:
 
    
statistics_by_match_local = Table(
    'statistics_by_match_local', meta, 
    Column('match_id', Integer, primary_key = True), 
    Column('team_id', Integer),    
    Column('local_player_0_id', Integer), 
    Column('local_player_0_rating', Float),    
    Column('local_player_1_id', Integer), 
    Column('local_player_1_rating', Float),    
    Column('local_player_2_id', Integer), 
    Column('local_player_3_rating', Float),    
    Column('local_player_4_id', Integer), 
    Column('local_player_4_rating', Float),    
    Column('local_player_5_id', Integer), 
    Column('local_player_5_rating', Float),    
    Column('local_player_6_id', Integer), 
    Column('local_player_6_rating', Float),    
    Column('local_player_7_id', Integer), 
    Column('local_player_7_rating', Float),    
    Column('local_player_8_id', Integer), 
    Column('local_player_8_rating', Float),    
    Column('local_player_9_id', Integer), 
    Column('local_player_9_rating', Float),    
    Column('local_player_10_id', Integer), 
    Column('local_player_10_rating', Float),    
)


statistics_by_match_away = Table(
    'statistics_by_match_away', meta, 
    Column('match_id', Integer, primary_key = True), 
    Column('team_id', Integer),    
    Column('away_player_0_id', Integer), 
    Column('away_player_0_rating', Float),    
    Column('away_player_1_id', Integer), 
    Column('away_player_1_rating', Float),    
    Column('away_player_2_id', Integer), 
    Column('away_player_3_rating', Float),    
    Column('away_player_4_id', Integer), 
    Column('away_player_4_rating', Float),    
    Column('away_player_5_id', Integer), 
    Column('away_player_5_rating', Float),    
    Column('away_player_6_id', Integer), 
    Column('away_player_6_rating', Float),    
    Column('away_player_7_id', Integer), 
    Column('away_player_7_rating', Float),    
    Column('away_player_8_id', Integer), 
    Column('away_player_8_rating', Float),    
    Column('away_player_9_id', Integer), 
    Column('away_player_9_rating', Float),    
    Column('away_player_10_id', Integer), 
    Column('away_player_10_rating', Float),    
)


In [48]:
meta.drop_all(conn)

In [49]:
meta.create_all(conn)

In [50]:
#Insert match data
for season in all_season_data:
    insert_data(conn,match2,all_season_data[season])

In [51]:
#Insert players data
for season in all_season_data:
    insert_data(conn,statistics,players_data[season])

In [52]:


#Insert ratings data
for year in seasons:
    matches=[]
    for match in range(0,len(all_season_data[year])):
        matches.append(all_season_data[year][match]["match_id"])
     
    for match in matches:
        auxLocal = ChainMap(*local)[match]
        auxAway = ChainMap(*away)[match]

        insert_data(conn, statistics_by_match_local,auxLocal[0])
        insert_data(conn, statistics_by_match_away,auxAway[0])


## Documentation
* <https://cloud.google.com/sdk/gcloud/reference/sql/connect>
* <https://docs.sqlalchemy.org/en/14/core/tutorial.html#deletes>
* <https://cloud.google.com/sql/docs/postgres/connect-app-engine-standard#private-ip_1>
* https://docs.sqlalchemy.org/en/14/core/metadata.html